# Chapter 10-04: CPU & NVMe Offloading

## 학습 목표
- GPU VRAM → CPU RAM → NVMe SSD의 **메모리 계층 구조**와 각 계층의 대역폭/지연 시간을 파악한다
- **ZeRO-Offload**의 핵심 아이디어: Adam 상태와 FP32 파라미터를 CPU로 내리고, GPU는 FP16 연산만 담당하는 분업 원리를 이해한다
- **PCIe 대역폭**이 실질적 병목으로 작용하는 시나리오를 계산하고, 파이프라인 오버랩으로 이를 극복하는 방법을 분석한다
- **Prefetch + Offload 오버랩** 전략이 전체 학습 속도에 미치는 영향을 수식으로 도출한다

## 목차
1. [수학적 기초: 메모리 계층과 PCIe 병목](#1.-수학적-기초)
2. [ZeRO-Offload 아키텍처](#2.-ZeRO-Offload-아키텍처)
3. [PCIe 대역폭 병목 계산](#3.-PCIe-병목-계산)
4. [파이프라인 오버랩 원리](#4.-파이프라인-오버랩)
5. [Offloading 효과 시뮬레이션](#5.-Offloading-시뮬레이션)
6. [Chapter 10 종합 정리](#6.-Chapter-10-종합정리)

## 1. 수학적 기초

### 메모리 계층 구조 (H100 기준)

| 계층 | 용량 | 대역폭 | 지연 시간 |
|------|------|--------|----------|
| **GPU HBM (VRAM)** | 80 GB | 3.35 TB/s | ~100 ns |
| **CPU DRAM (RAM)** | 512 GB~수 TB | ~200 GB/s | ~100 ns |
| **NVMe SSD** | 수 TB~수십 TB | ~7 GB/s | ~100 µs |
| **PCIe 4.0 ×16** (GPU↔CPU) | — | ~64 GB/s | ~수 µs |

### ZeRO-Offload 메모리 절약 공식

CPU로 내리는 항목: FP32 마스터 가중치 + Adam $m$, $v$ = $4P + 4P + 4P = 12P$ bytes  
GPU에 남는 것: FP16 파라미터 + FP16 그래디언트 = $2P + 2P = 4P$ bytes

$$\boxed{M_{GPU}^{\text{Offload}} = 4P \text{ bytes} \quad (\text{Baseline } 16P \text{ 대비 } 75\% \text{ 절약})}$$

### PCIe 병목 분석

Offloading 시 PCIe로 오가는 데이터 (스텝당):
- GPU→CPU: FP16 그래디언트 $2P$ bytes (Backward 후)
- CPU→GPU: FP16 업데이트된 파라미터 $2P$ bytes (Adam 업데이트 후)

$$T_{PCIe} = \frac{4P}{BW_{PCIe}}$$

GPU 연산 시간: $T_{GPU} \approx \frac{\text{FLOPs}}{T_{FLOPS}}$

병목 조건: $T_{PCIe} > T_{GPU}$ → Offloading이 학습 속도를 제한!

---

### 🐣 초등학생을 위한 친절 설명!

#### 📦 CPU Offloading이 뭔가요?

> 책상(GPU) 위에 너무 많은 책(Adam 상태)이 있으면 정작 공부가 안 되잖아요!  
>
> **해결책**: 자주 안 쓰는 책들을 옆 선반(CPU 메모리)에 저장!
> - 필요할 때만 선반에서 가져오기 (PCIe 전송)
> - 공부(GPU 연산)하는 동안 다음에 필요한 책 미리 가져오기 (Prefetch!)
>
> 문제: 선반까지 왔다갔다 하는 시간(PCIe 대역폭)이 너무 오래 걸리면 안 되겠죠?

### 📝 연습 문제

#### 문제 1: Offloading 후 GPU 메모리

70B 모델에 ZeRO-Offload 적용 (Adam 상태 + 마스터 가중치 CPU 이동) 시  
GPU당 메모리는? (단일 GPU 시나리오)

<details>
<summary>💡 풀이 확인</summary>

$P = 70$B

- CPU로 이동: FP32 마스터($4P$) + Adam $m$($4P$) + Adam $v$($4P$) = $12P$
- GPU 유지: FP16 모델($2P$) + FP16 그래디언트($2P$) = $4P$

$$M_{GPU} = 4 \times 70 = 280 \text{ GB}$$

**여전히 H100 80GB 초과!** → ZeRO Stage 3와 결합 필요

ZeRO3 + Offload: $4P/N + \text{Activation}$ → $4 \times 70/8 = 35$ GB (N=8) ✅
</details>

#### 문제 2: PCIe 병목 여부 판단

7B 모델, H100(312 TFLOPS FP16), PCIe 4.0 ×16(64 GB/s) 환경에서  
스텝당 PCIe 전송 시간 vs GPU 연산 시간 비교 (시퀀스 길이 2048, 배치 4 기준)

<details>
<summary>💡 풀이 확인</summary>

PCIe 전송량 = $4P$ bytes = $4 \times 7 \times 10^9 = 28$ GB  
$T_{PCIe} = 28 / 64 \approx 0.44$ 초

GPU FLOPs = $6P \times B \times S \approx 6 \times 7\times10^{9} \times 4 \times 2048 \approx 344$ PFLOPS  
$T_{GPU} = 344 \times 10^{15} / (312 \times 10^{12}) \approx 1.1$ 초

$T_{PCIe} < T_{GPU}$ → 이 경우 PCIe가 병목이 아님 (오버랩 가능!) ✅
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch

np.random.seed(42)

## 2. ZeRO-Offload 아키텍처

In [ ]:
# ---------------------------------------------------
# ZeRO-Offload 아키텍처 다이어그램
# ---------------------------------------------------

fig, ax = plt.subplots(figsize=(13, 7))
ax.set_xlim(0, 13); ax.set_ylim(0, 8)
ax.axis('off')
ax.set_title('ZeRO-Offload 아키텍처: GPU ↔ CPU 역할 분담', fontsize=13, fontweight='bold')

# GPU 박스
gpu_box = FancyBboxPatch((0.5, 3.0), 5.5, 4.5, boxstyle='round,pad=0.2',
                          facecolor='#E3F2FD', edgecolor='#1565C0', lw=2.5)
ax.add_patch(gpu_box)
ax.text(3.25, 7.2, '🖥️  GPU (H100 80GB)', ha='center', fontsize=12, fontweight='bold', color='#1565C0')

# GPU 내부 - 보존 항목
for i, (label, color) in enumerate([
    ('FP16 Model Params (2P)', '#42A5F5'),
    ('FP16 Gradients (2P)', '#66BB6A'),
    ('Activation Memory', '#FFA726'),
]):
    rect = FancyBboxPatch((0.8, 3.3 + i*1.1), 5.0, 0.9,
                          boxstyle='round,pad=0.05', facecolor=color, alpha=0.75, edgecolor='white', lw=1.5)
    ax.add_patch(rect)
    ax.text(3.3, 3.75 + i*1.1, label, ha='center', fontsize=10, color='white', fontweight='bold')

ax.text(3.25, 3.1, f'총 ≈ 4P + Act  (Baseline 대비 75% 절약!)', ha='center', fontsize=9, color='#1565C0')

# CPU 박스
cpu_box = FancyBboxPatch((7.0, 3.0), 5.5, 4.5, boxstyle='round,pad=0.2',
                          facecolor='#FFF3E0', edgecolor='#E65100', lw=2.5)
ax.add_patch(cpu_box)
ax.text(9.75, 7.2, '🧠  CPU RAM (≥512 GB)', ha='center', fontsize=12, fontweight='bold', color='#E65100')

# CPU 내부 - Offload 항목
for i, (label, color) in enumerate([
    ('FP32 Master Weights (4P)', '#EF5350'),
    ('Adam m: 1st Moment (4P)', '#AB47BC'),
    ('Adam v: 2nd Moment (4P)', '#7B1FA2'),
]):
    rect = FancyBboxPatch((7.3, 3.3 + i*1.1), 5.0, 0.9,
                          boxstyle='round,pad=0.05', facecolor=color, alpha=0.75, edgecolor='white', lw=1.5)
    ax.add_patch(rect)
    ax.text(9.8, 3.75 + i*1.1, label, ha='center', fontsize=10, color='white', fontweight='bold')

ax.text(9.75, 3.1, '총 = 12P  (Offloaded to CPU!)', ha='center', fontsize=9, color='#E65100')

# PCIe 화살표
ax.annotate('FP16 Grad →\n(Backward 후)', xy=(7.0, 5.5), xytext=(6.0, 5.8),
            arrowprops=dict(arrowstyle='->', color='#43A047', lw=2.5),
            fontsize=9, ha='center', color='#43A047')
ax.annotate('← FP16 Param 업데이트\n(Adam 후)', xy=(6.0, 4.5), xytext=(6.0, 4.5),
            fontsize=8, ha='center', color='#1565C0')
ax.annotate('', xy=(6.0, 4.2), xytext=(7.0, 4.5),
            arrowprops=dict(arrowstyle='->', color='#1565C0', lw=2.5))

ax.text(6.25, 6.3, 'PCIe 4.0\n64 GB/s', ha='center', fontsize=9, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='orange', alpha=0.9))

# NVMe
nvme_box = FancyBboxPatch((3.5, 0.3), 6.0, 1.5, boxstyle='round,pad=0.2',
                           facecolor='#EFEBE9', edgecolor='#4E342E', lw=2)
ax.add_patch(nvme_box)
ax.text(6.5, 1.5, '💾 NVMe SSD (ZeRO-Infinity)', ha='center', fontsize=11, fontweight='bold', color='#4E342E')
ax.text(6.5, 0.8, 'Activation / Checkpoint → 7 GB/s  (ZeRO-Infinity extension)', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print("ZeRO-Offload 핵심:")
print("  GPU: FP16 연산 전문 (빠른 Tensor Core 활용)")
print("  CPU: Adam 업데이트 전담 (느리지만 큰 메모리 활용)")
print("  PCIe: 양방향 전송 (그래디언트 UP → Adam UP → 파라미터 DOWN)")

## 3. PCIe 병목 계산

In [ ]:
# ---------------------------------------------------
# PCIe 병목 분석: 모델/배치 크기별 계산
# ---------------------------------------------------

def analyze_bottleneck(params_B, batch_size, seq_len,
                        pcie_bw_GBps=64,       # PCIe 4.0 ×16
                        gpu_tflops=312):        # H100 FP16
    """
    PCIe 전송 시간과 GPU 연산 시간 비교.
    T_pcie > T_gpu: PCIe가 병목 (Offloading이 느림)
    T_pcie < T_gpu: 오버랩 가능 (Offloading 효율적)
    """
    P = params_B * 1e9

    # PCIe 전송량: Grad(FP16) + 업데이트된 Param(FP16)
    pcie_bytes = 4 * P  # 2P send + 2P recv
    T_pcie = pcie_bytes / (pcie_bw_GBps * 1e9)

    # Transformer FLOPs 근사: 6P × B × S (Forward+Backward)
    flops = 6 * P * batch_size * seq_len
    T_gpu = flops / (gpu_tflops * 1e12)

    ratio = T_pcie / T_gpu
    bottleneck = '🔴 PCIe 병목!' if ratio > 1 else '✅ 오버랩 가능'

    return T_pcie, T_gpu, ratio, bottleneck


print(f"{'모델':>12} | {'B':>3} | {'S':>5} | {'T_PCIe(s)':>10} | {'T_GPU(s)':>10} | {'비율':>7} | 상태")
print("-" * 72)

configs = [
    (7,  1, 512),   (7,  4, 2048),  (7,  8, 4096),
    (70, 1, 512),   (70, 4, 2048),  (70, 8, 4096),
    (175,1, 512),   (175,4, 2048),
]

for P, B, S in configs:
    tp, tg, r, b = analyze_bottleneck(P, B, S)
    print(f"{f'{P}B':>12} | {B:>3} | {S:>5} | {tp:>10.3f} | {tg:>10.3f} | {r:>7.2f}x | {b}")

## 4. 파이프라인 오버랩

In [ ]:
# ---------------------------------------------------
# Prefetch + Offload 파이프라인 오버랩 시각화
# ---------------------------------------------------

def draw_pipeline_timeline(ax, T_gpu, T_pcie, n_steps=4):
    """오버랩 없음 vs 파이프라인 오버랩 비교"""
    colors = {'GPU': '#1E88E5', 'PCIe': '#E53935', 'CPU': '#43A047'}

    # ── 위: 오버랩 없음 ──────────────────────────────
    t = 0
    for step in range(n_steps):
        for phase, dur, color_key in [
            ('GPU\n연산', T_gpu, 'GPU'),
            ('PCIe\n전송', T_pcie, 'PCIe'),
            ('CPU\nAdam', T_pcie * 0.5, 'CPU'),
            ('PCIe\n복원', T_pcie, 'PCIe'),
        ]:
            rect = FancyBboxPatch((t, 1.1), dur * 50, 0.8,
                                  boxstyle='round,pad=0.02',
                                  facecolor=colors[color_key], alpha=0.75, edgecolor='white', lw=1)
            ax.add_patch(rect)
            ax.text(t + dur * 25, 1.5, phase, ha='center', va='center', fontsize=7, color='white', fontweight='bold')
            t += dur * 50

    total_naive = t

    # ── 아래: 파이프라인 오버랩 ─────────────────────
    t = 0
    gpu_end = 0
    for step in range(n_steps):
        gpu_start = t
        rect = FancyBboxPatch((gpu_start, 0.0), T_gpu * 50, 0.8,
                              boxstyle='round,pad=0.02',
                              facecolor=colors['GPU'], alpha=0.75, edgecolor='white', lw=1)
        ax.add_patch(rect)
        ax.text(gpu_start + T_gpu * 25, 0.4, f'GPU\nStep{step+1}', ha='center', va='center',
                fontsize=7, color='white', fontweight='bold')

        # PCIe를 GPU와 오버랩
        if step > 0:
            pcie_start = gpu_start
            rect2 = FancyBboxPatch((pcie_start, -0.9), T_pcie * 50, 0.8,
                                   boxstyle='round,pad=0.02',
                                   facecolor=colors['PCIe'], alpha=0.75, edgecolor='white', lw=1)
            ax.add_patch(rect2)
            ax.text(pcie_start + T_pcie * 25, -0.5, 'PCIe', ha='center', va='center',
                    fontsize=7, color='white', fontweight='bold')

        t += T_gpu * 50

    total_pipeline = t

    # 축 설정
    ax.set_xlim(0, max(total_naive, total_pipeline) * 1.05)
    ax.set_ylim(-1.2, 2.3)
    ax.set_yticks([1.5, 0.4, -0.5])
    ax.set_yticklabels(['오버랩\n없음', 'GPU\n연산', 'PCIe\n전송'], fontsize=9)
    ax.axvline(x=total_naive, color='red', ls='--', lw=1.5, alpha=0.6, label=f'No-Overlap: {total_naive:.0f}')
    ax.axvline(x=total_pipeline, color='blue', ls='--', lw=1.5, alpha=0.6, label=f'Pipeline: {total_pipeline:.0f}')
    ax.set_xlabel('시간 (상대값)')
    ax.legend(fontsize=9)


T_gpu   = 1.0   # GPU 연산 = 1 단위
T_pcie  = 0.4   # PCIe 전송 = 0.4 단위 (오버랩 효율적인 케이스)

fig, ax = plt.subplots(figsize=(14, 5))
draw_pipeline_timeline(ax, T_gpu, T_pcie)
ax.set_title('Prefetch + Offload 파이프라인 오버랩 (T_PCIe < T_GPU 케이스)', fontweight='bold')
plt.tight_layout()
plt.show()

n = 4
naive_total = n * (T_gpu + T_pcie * 2 + T_pcie * 0.5)
pipeline_total = T_pcie * 2 + n * T_gpu  # 파이프라인 안정 상태
speedup = naive_total / pipeline_total
print(f"오버랩 없음 총 시간: {naive_total:.2f} 단위")
print(f"파이프라인 총 시간:  {pipeline_total:.2f} 단위")
print(f"속도 향상:           {speedup:.2f}배")

## 5. Offloading 시뮬레이션

In [ ]:
# ---------------------------------------------------
# Offloading 효과 종합 비교 시각화
# ---------------------------------------------------

# 메모리 절약 기법 비교 (70B 모델, 8-GPU 기준)
P = 70
N = 8
gpu_limit = 80 * 0.7

methods = [
    ('Baseline DP',           16 * P),
    ('ZeRO Stage 1',          4*P + 8*P/N),
    ('ZeRO Stage 2',          2*P + 6*P/N),
    ('ZeRO Stage 3',          16*P/N),
    ('ZeRO Stage 3\n+ CPU Offload', 4*P/N),  # FP16만 GPU에
    ('ZeRO Stage 3\n+ NVMe Offload', 2*P/N), # Param도 NVMe로
]

fig, ax = plt.subplots(figsize=(12, 6))
colors_m = ['#E53935', '#FB8C00', '#FDD835', '#43A047', '#1E88E5', '#5E35B1']
vals = [m for _, m in methods]
bars = ax.barh(range(len(methods)), vals,
               color=colors_m, alpha=0.85, height=0.6, edgecolor='white', lw=1.5)

ax.axvline(x=gpu_limit, color='k', ls='--', lw=2.5, label=f'H100 {gpu_limit:.0f}GB 한계')

for i, (bar, (label, val)) in enumerate(zip(bars, methods)):
    reduction = (1 - val / vals[0]) * 100
    ok = '✅' if val <= gpu_limit else '❌'
    text = f'{val:.0f} GB {ok}' if i == 0 else f'{val:.0f} GB (-{reduction:.0f}%) {ok}'
    ax.text(val + 3, i, text, va='center', fontsize=9, fontweight='bold')

ax.set_yticks(range(len(methods)))
ax.set_yticklabels([m[0] for m in methods], fontsize=9)
ax.set_xlabel('GPU당 모델 메모리 (GB)', fontsize=11)
ax.set_title(f'Llama-2 70B 메모리 최적화 기법 전체 비교\n(N={N} GPU, H100 80GB)', fontweight='bold')
ax.set_xlim(0, vals[0] * 1.2)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 6. Chapter 10 종합정리

### Chapter 10 전체 기법 비교

| 기법 | 절약 대상 | 메모리 절약 | 연산 오버헤드 | 통신 오버헤드 |
|------|----------|-----------|------------|------------|
| **Gradient Checkpointing** | Activation | $O(L) \to O(\sqrt{L})$ | +33% FLOPs | 없음 |
| **ZeRO Stage 1** | Optimizer States | ~50% | 없음 | Scatter 추가 |
| **ZeRO Stage 2** | + Gradients | ~75% | 없음 | Reduce-Scatter |
| **ZeRO Stage 3** | + Parameters | $1/N$ | 없음 | All-Gather 2회 |
| **CPU Offload** | Optimizer+Master | 75% (GPU 내) | CPU 연산 시간 | PCIe 전송 |
| **NVMe Offload** | + Activation | 거의 전부 | Prefetch 복잡도 | PCIe×느림 |

### 실전 추천 조합

```
모델 크기  GPU 수  권장 조합
───────────────────────────────────────────────────────
~7B         ≥8   Mixed Precision + ZeRO Stage 1
~70B        ≥8   Mixed Precision + ZeRO Stage 3 + GC
~175B       ≥64  Mixed Precision + ZeRO Stage 3 + 3D PP
~530B+      ≥256 + CPU Offload or NVMe (ZeRO-Infinity)
```

### 다음 챕터 예고
**Chapter 11: 커스텀 GPU 커널 프로그래밍** — 메모리 최적화의 마지막 단계: 연산 자체를 GPU에서 최대 효율로 수행하는 커스텀 CUDA 커널과 Triton 프로그래밍. Arithmetic Intensity, Kernel Fusion, Shared Memory 활용을 학습한다.